In [1]:
pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.6/222.6 KB 5.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 33.4 MB/s eta 0:00:0000:0100:01
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl (181 kB)
  Using cached rsa-4.9.1-py3-none-any.whl (34 kB)
  Using cached cachetools-6.2.1-py3-none-any.whl (11 kB)
  Using cached google_auth-2.41.1-py2.py3-none-any.whl (221 kB)
  Using cached httplib2-0.31.0-py3-none-any.whl (91 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.7/173.7 KB 21.5 MB/s eta 0:00:00
  Using cached uritemplate-4.2.0-py3-none-any.whl (11 kB)
  Using cached requests-2.32.5-py3-none-any.whl (64 kB)
  Using cached googleapis_common_protos-1.71.0-py3-none-any.whl (294 kB)
  Using cached protobuf-6.33.0-cp39-abi3-macosx_10_9_universal2.whl (427 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl (50 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl (83 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 KB 22.9 MB/s eta 0

In [25]:
import os
import json
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


In [8]:
# Load env vars
CREDENTIALS_PATH = "/Users/joelsng/Documents/GitHub/levelsliving-IMS/server/google_credentials.json"
SPREADSHEET_ID = "1ny7RVODq6twymuqxWZDzAV8w0jHJWjQT6TrnJl5VpII"

# Authenticate using service account
creds = service_account.Credentials.from_service_account_file(
    CREDENTIALS_PATH,
    scopes=["https://www.googleapis.com/auth/spreadsheets"]
)
service = build("sheets", "v4", credentials=creds)

In [9]:
print(SPREADSHEET_ID)

1ny7RVODq6twymuqxWZDzAV8w0jHJWjQT6TrnJl5VpII


In [21]:
SHEET_NAME = "OrdersArchive"  
RANGE = f"{SHEET_NAME}!A1:J"  # 10 columns (A:J), all rows

In [22]:
try:
    resp = service.spreadsheets().values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE,
        valueRenderOption="UNFORMATTED_VALUE",     # raw numbers/dates
        dateTimeRenderOption="FORMATTED_STRING"    # keep date/time as strings if present
    ).execute()

    rows = resp.get("values", [])
    if not rows:
        df = pd.DataFrame(columns=[
            "Order Date","SKU","Item","Variant","Quantity","Value","Delivery Date","Delivery Time"
        ])
    else:
        header, data = rows[0], rows[1:]
        df = pd.DataFrame(data, columns=header[:len(data[0])]) if data else pd.DataFrame(columns=header)

    print(df.shape, "rows x cols")
except HttpError as e:
    print("Sheets read failed:", e)

(18217, 8) rows x cols


In [20]:
df

,Order Date,SKU,Item,Variant,Quantity,Value,Delivery Date,Delivery Time
0,2025-10-10 05:08:43+00:00,B16LD,Storage Bedframe (No headboard),Servicing,1.0,775.28,2025-10-24,2025-10-24
1,2024-12-27 04:38:47+00:00,DC1606/RusticWH,Verona 1.6m 3x2 Dresser Sideboard,Assembly,1.0,773,2025-07-09,2025-07-09
2,2024-12-27 04:38:47+00:00,DC8036/RusticWH,Union 80cm Chest of 6 Drawers,Assembly,1.0,,2025-07-09,2025-07-09
3,2025-01-28 07:26:02+00:00,CTM10909,Custom Furniture,Custom,1.0,,2025-08-14,2025-08-14
4,2025-01-28 07:26:02+00:00,CTM10909,Custom Furniture,Custom,1.0,,2025-08-14,2025-08-14
...,...,...,...,...,...,...,...,...
18212,2025-09-05 10:28:00+00:00,SC5,Vegas 1.2m Oak Shoe Cabinet,,1.0,None,None,None
18213,2025-09-05 10:28:00+00:00,CU,Soft Close Hinge Upgrade,Soft Close,1.0,None,None,None
18214,2025-09-05 15:17:00+00:00,SC5,Vegas 1.2m Oak Shoe Cabinet,,1.0,None,None,None
18215,2025-09-05 18:04:00+00:00,B16LD,Winter Storage Bedframe,Plush Fabric,1.0,None,None,None


In [33]:
SHEET_NAME = "Orders"  
cols = ["A", "B", "C", "D", "H", "I", "K", "R", "T", "Y"]
ranges = [f"{SHEET_NAME}!{col}1:{col}" for col in cols]

In [35]:
# Batch request (all in one call)
resp = service.spreadsheets().values().batchGet(
    spreadsheetId=SPREADSHEET_ID,
    ranges=ranges,
    valueRenderOption="UNFORMATTED_VALUE",
    dateTimeRenderOption="FORMATTED_STRING"
).execute()

In [37]:
# Each range returns a column of values
columns_data = [v.get("values", [[]]) for v in resp.get("valueRanges", [])]
# Normalize all columns to same length
max_len = max(len(c) for c in columns_data)
normalized = [(c + [[]]*(max_len-len(c))) for c in columns_data]
# Extract column names from first row
headers = [c[0][0] if c and c[0] else f"Col_{i}" for i, c in enumerate(columns_data)]


In [40]:
data = list(zip(*[ [r[0] if r else None for r in col[1:]] for col in normalized ]))
df = pd.DataFrame(data, columns=headers)
df.head()

,Order No,Order Date,Customer Name,Customer Contact,SKU,Item,Quantity,Delivered?,Remarks,Order Value
0,S12123,2025-10-10T13:08:43+08:00,Yang -,+6597418736,B16LD,Storage Bedframe (No headboard),1,TRUE,None,775.28
1,10666,2024-12-27T12:38:47+08:00,Adeline Ng,+6597972142,DC1606/RusticWH,Verona 1.6m 3x2 Dresser Sideboard,1,TRUE,Hold Delivery PO34A Regular - Ready,773
2,10666,2024-12-27T12:38:47+08:00,Adeline Ng,+6597972142,DC8036/RusticWH,Union 80cm Chest of 6 Drawers,1,TRUE,Hold Delivery PO34A Regular - Ready,None
3,10909,2025-01-28T15:26:02+08:00,Mario Morales,+6596952066,CTM10909,Custom Furniture,1,TRUE,PO35B Custom - Ready,None
4,10909,2025-01-28T15:26:02+08:00,Mario Morales,+6596952066,CTM10909,Custom Furniture,1,TRUE,PO35B Custom - Ready,None


In [ ]:
import os
import csv
from google.oauth2 import service_account
from googleapiclient.discovery import build

col_map = {
    "Order Date": "B",
    "SKU": "H",
    "Item": "I",
    "Variant": "J",
    "Quantity": "K",
    "Value": "Y",
}

creds = service_account.Credentials.from_service_account_file(
    CREDENTIALS_PATH, scopes=["https://www.googleapis.com/auth/spreadsheets.readonly"]
)
service = build("sheets", "v4", credentials=creds)

ranges = [f"{SHEET_NAME}!{col}1:{col}" for col in col_map.values()]
resp = service.spreadsheets().values().batchGet(
    spreadsheetId=SPREADSHEET_ID,
    ranges=ranges,
    valueRenderOption="UNFORMATTED_VALUE",
    dateTimeRenderOption="FORMATTED_STRING",
).execute()

value_ranges = resp.get("valueRanges", [])
# Build columns in the requested order
columns = list(col_map.keys())
cols_data = [vr.get("values", []) for vr in value_ranges]
max_len = max((len(c) for c in cols_data), default=0)

# Normalize columns to equal length
for i, c in enumerate(cols_data):
    if len(c) < max_len:
        cols_data[i] = c + [[]] * (max_len - len(c))

# Row-wise compose (skip header row index 0; we'll use our own headers)
rows = []
for row_idx in range(1, max_len):  # start at 1 to skip the sheet's first row (assumed header)
    row = []
    for col in cols_data:
        cell = col[row_idx][0] if (row_idx < len(col) and col[row_idx]) else ""
        row.append(cell)
    rows.append(row)

out_dir = os.path.join(os.path.dirname(__file__), "tmp")
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, "sheets_orders.csv")

with open(out_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(columns)  # our normalized header
    writer.writerows(rows)
